In [1]:
import pandas as pd
import numpy as np

In [2]:
def separa_dataframe(df: pd.DataFrame, grandezza_frammento):
    frammenti = []

    # Frammento il dataframe
    for i in range(0, len(df), grandezza_frammento):
        frammenti.append(df.iloc[i : i + grandezza_frammento])
    
    # Rimuovo l'ultimo elemento che risulta in un dataframe di una riga
    frammenti.pop()

    return frammenti

In [3]:
def variazione_media(ndarray_bmi):
    # Calcolo la differenza tra gli elementi consecutivi dell'array
    differenze = np.diff(ndarray_bmi)
    # Calcolo la media delle differenze
    variazione_media = np.mean(differenze)
    
    return round(variazione_media, 3)

In [4]:
def pulisci_lista_bmi(lista_bmi):
    # Converto la lista in un array di nunpy
    array = np.array(lista_bmi)
    # Prendo solo gli elementi non NaN dalla lista
    array = array[~np.isnan(array)]

    return np.array(array)

In [5]:
def pulisci_lista_steps(lista_steps):
    # Converto la lista in un array di nunpy
    array = np.array(lista_steps)
    # Calcolo il valore Medio
    media = np.mean(array[~np.isnan(array)])
    # Metto i valori medi al posto dei NaN
    array[np.isnan(array)] = media
    
    return array

In [6]:
def main():

    # Ottengo le informazioni dai CSV
    dataset = pd.read_csv('../dataset.csv')
    utenti = pd.read_csv('../infoutenti.csv')

    dataset_cbr = pd.DataFrame(columns=['steps_bins', 'bmi_variation', 'action_bin'])

    for utente in utenti['utenti']:
        
        # Estraggo le informazioni dell'utente
        df_utente: pd.DataFrame = dataset[['bins_passi', 'bmi']].loc[dataset['utente'] == utente]

        # Se il dataset non contiene informazioni sull'utente passa a quello successivo
        if df_utente.empty:
            continue

        print('utente: {}'.format(utente))
        
        # Spezzo il Dataframe
        frammenti_df_utente = separa_dataframe(df_utente, 7)

        # Creo le righe come servono per il CBR
        mini_df_utente: pd.DataFrame
        for mini_df_utente in frammenti_df_utente:

            # Valido il BMI
            if (mini_df_utente['bmi'].isnull().sum() > 5):
                continue
            else:
                bmi_list = pulisci_lista_bmi(mini_df_utente['bmi'])

            # Valido gli steps
            if (mini_df_utente['bins_passi'].isnull().sum() > 2):
                continue
            elif (mini_df_utente['bins_passi'].isnull().sum() <= 2):
                steps_bins_list = pulisci_lista_steps(mini_df_utente['bins_passi'])
            else:
                steps_bins_list = np.array(mini_df_utente['bins_passi'])

            # prendo i bin e creo lo stato
            steps_bins = [int(x) for x in steps_bins_list[:6].tolist()]

            # Prendo il BMI e ne calcolo la variazione media
            bmi_variation = variazione_media(bmi_list)

            # prendo l'ultimo elemento del mini dataframe che funziona da azione
            action_bin = int(steps_bins_list[-1])

            # aggiungo le righe al dataframe finale
            nuova_riga = {'steps_bins': steps_bins, 'bmi_variation': bmi_variation, 'action_bin': action_bin}
            dataset_cbr = pd.concat([dataset_cbr, pd.DataFrame([nuova_riga])], ignore_index = True)

    # Verifiche Preliminari del Dataset
    display('Il dataset contiene {} casi\n'.format(dataset_cbr.index.size))
    display(dataset_cbr.head())
    dataset_cbr.to_csv('dataset_cbr.csv', index=False)

In [7]:
main()

utente: d9874c86e6db7bf4c3959445b0b05a1bb8e60bbf
utente: fe5d7aacd82682e68d94ea1e6aa33a1e858337f7
utente: 88359b8e24f4d6b758fb4566860ba9842fc5b350
utente: e061ccf5eece921027f618686a5aa2412d753b6b
utente: 2e88aac4e69049d0845a5e2dce661ae8f4f55239
utente: b4e87340761e4c917db734cd52f56ed9abd0b314
utente: fe47edd67278f09ed33be57b5008adbbaf5cb054
utente: 5de5c9b6cf14de18385ed689169cd99e6c379ad9
utente: fab1fa81eaf02d1c3bc0731fd5a2f576d1498b01
utente: ff32ef9b713fb27c972e8d1916680dd0ccebf646
utente: 1c5ddfde99185d10c666b1e3c95e6559cc00ba3c
utente: 23d97494986b76813a441ce44e1d9804b0a68032
utente: 23e4462c79b4ed02a8c6c9ec662431c585f63ff8
utente: bec3c582e5fee444f82a057f6acee424f34a06cd
utente: 328221ecda99a9fc3c72ae5cf34243df7f2d23c7
utente: 93ec1b10677831f79ad43ae6b97ddd0547a4c5ee
utente: 18f6a7f626be651bbf20a98e53b4700a9a17c65d
utente: b5cd5af4a8eb352dc23d182f50e6fa3fe6538451
utente: 87ffe28e9fc5c2117efef20edb3904872f8a1d68
utente: b10f6064b4e8d90d23d0eb3bf3591d80fb4ee245
utente: 35b7ca289ebf

'Il dataset contiene 69435 casi\n'

,steps_bins,bmi_variation,action_bin
0,"[5, 1, 5, 4, 2, 2]",-0.007,1
1,"[4, 5, 3, 2, 4, 4]",-0.018,1
2,"[3, 5, 5, 4, 3, 4]",-0.015,3
3,"[5, 5, 5, 5, 2, 1]",-0.005,3
4,"[3, 2, 2, 4, 3, 2]",0.024,3
